In [2]:
import matplotlib.pyplot as plt 
import numpy as np
#Activar para que puedan correr los codigos :)

In [2]:
#Punto 4
def mul_matrices(A,B):
    A = np.array([[-2, 1, 0], [1, -2, 1], [0, 1, -2]])
    B = np.array([-2, 1, 0])

    n = np.shape(A)[1]
    n2 = np.shape(B)[0]
    n3 = np.shape(B)[0]
    n4 = np.shape(A)[0]
    m = np.zeros(shape=(n4, n2))

    if n == n3:
        for j in range(n4):
            for i in range(n2):
                for k in range(n3):
                    m[j][i] += A[j][k] * B[k]
        return m
    else:
        return "no se pueden multiplicar las matrices"

In [4]:
#Punto 7
def sor(A,b,x0,w,itmax=1000,tolerancia=1e-14):
    x=x0.copy()
    sum_k= x.copy()
    residuo = np.linalg.norm( np.dot(A,x) - b)
    it=0
    while residuo>= tolerancia and it < itmax:
    
        sum_k[:]=0
        
        for i in range (A.shape[0]):
            for j in range (A.shape[1]):
                if i != j:
                     sum_k[i] += A[i][j] * x[j]
            x[i] = (1 - w) * x[i] + (w / A[i][i]) * (b[i] - sum_k[i])
        
        residuo = np.linalg.norm(np.dot(A, x) - b)
        it += 1
        
        if residuo > 1000:
            print('No calculable con SOR')
            x[:] = 0.
            break
    
        
    return x,it
def omega_optimo(A,b,x0,itmax=1000,tolerancia=1e-14):
    w=None
    iteracion_minima=float('inf')
    for w0 in range(1,2):
        metodo= sor(A,b,x0,itmax,tolerancia)
        iteracion=len(metodo)
        
        if iteracion< iteracion_minima:
            iteracion_minima=iteracion
            w=w0   
    return w
A = np.array([[3,-1,-1],[-1.,3.,1.],[2,1,4]])
b = np.array([1.,3.,7.])
w=omega_optimo(A,b,np.array([0,0,0]))
x=sor(A,b,np.array([0,0,0]),w)
x

No calculable con SOR


(array([1, 1, 1]), 2)

In [15]:
# punto 11 

H= np.array([[1,2,-1],[1,0,1],[4,-4,5]])


def autovalor_min (m):
    W = np.linalg.inv(m)
    
    q = np.random.rand(len(W))
    n=np.shape(W)[0]
    
    q1=np.zeros(shape=(n,1))
    for i in range(len(q)):
        q1[i][0]= q[i]
        
    for i in range(100):
        z= np.dot(W, q1)
        q1= z/np.sqrt(np.dot(z.T,z))
        valor_min= np.dot(q1.T,W)
        valor_min= np.dot(valor_min, q1)
    return "El valor de estado base E0 es "+ str(1/valor_min) + " y su vector propio es " + str(q1).replace("\n", " ")
autovalor_min(H)


'El valor de estado base E0 es [[1.]] y su vector propio es [[-0.40824829]  [ 0.40824829]  [ 0.81649658]]'

In [11]:
#Punto 12a

G1= np.array([lambda x,y:(np.log(x**2+y**2)-np.sin(x*y))-(np.log(2)+np.log(np.pi)), \
              lambda x,y: np.exp(x-y)+np.cos(x*y)])

# Por descenso al gradiente 
def funcion_1(x,y):
    f1=((np.log(x**2+y**2)-np.sin(x*y))-(np.log(2)+np.log(np.pi)))
    f2=(np.exp(x-y)+np.cos(x*y))
    return f1**2+f2**2

Dx = lambda f,x,y,h=1e-5: (f(x+h,y) - f(x-h,y))/(2*h)
Dy = lambda f,x,y,h=1e-5: (f(x,y+h) - f(x,y-h))/(2*h)
x0, y0 = 0.5,0.1
Gradient = lambda f,x,y: np.array([Dx(f,x,y),Dy(f,x,y)])
def Minimizer(f, N = 100, gamma = 0.01):
    
    r = np.zeros((N,2))
    r[0] = np.random.uniform(-5.,5.,size=2)
    
    Grad = np.zeros((N,2))
    Grad[0] = Gradient(f,r[0,0],r[0,1])
    
    for i in (range(1,N)):
        r[i] = r[i-1] - gamma*Gradient(f,r[i-1,0],r[i-1,1])
        Grad[i] = Gradient(f,r[i-1,0],r[i-1,1])
        
        
    return r,Grad
N = 200
r,Grad = Minimizer(funcion_1,N)
print(r[-1])

#por Newton-Raphson
G1= np.array([lambda x,y:(np.log(x**2+y**2)-np.sin(x*y))-(np.log(2)+np.log(np.pi)),
              lambda x,y: np.exp(x-y)+np.cos(x*y)])

def GetF(G,r):
    
    n = r.shape[0]
    
    v = np.zeros_like(r)
    
    for i in range(n):
        v[i] = G[i](r[0],r[1])
        
    return v
def GetJacobian(f,r,h=1e-6):
    
    n = r.shape[0]
    
    J = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            
            rf = r.copy()
            rb = r.copy()
            
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            
            J[i,j] = ( f[i](rf[0],rf[1]) - f[i](rb[0],rb[1])  )/(2*h)
            
    
    return J

def NewtonRaphson(G,r,itmax=1000,error=1e-9):
    
    it = 0
    d = 1.
    dvector = []
    
    while d > error and it < itmax:
        
        # Vector actual
        rc = r
        
        F = GetF(G,rc)
        J = GetJacobian(G,rc)
        InvJ = np.linalg.inv(J)
        
        r = rc - np.matmul(InvJ,F)
        
        diff = r - rc
        
        d = np.max( np.abs(diff) )
        
        dvector.append(d)
        it += 1
    
    return r,dvector


r2,dvector = NewtonRaphson(G1,np.array([2.,2.]))
print(r2)


[-2.30031595  4.7717374 ]
[1.77245385 1.77245385]


In [45]:
#Punto 12b 
from IPython.display import clear_output
import time
G2 = np.array([lambda x,y,z: 6*x - 2*np.cos(y*z) - 1.,\
     lambda x,y,z: 9*y + np.sqrt( x**2 + np.sin(z) + 1.06 ) + 0.9,\
     lambda x,y,z: 3*x +6*y+6*z])

#Usando Newton Rhapson para este sistema 


def GetF(G,r):
    
    n = r.shape[0]
    
    v = np.zeros_like(r)
    
    for i in range(n):
        v[i] = G[i](r[0],r[1],r[2])
        
    return v
def GetJacobian(f,r,h=1e-6):
    
    n = r.shape[0]
    
    J = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            
            rf = r.copy()
            rb = r.copy()
            
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            
            J[i,j] = ( f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2])  )/(2*h)
            
    
    return J
def NewtonRaphson(G,r,itmax=1000,error=1e-9):
    
    it = 0
    d = 1.
    dvector = []
    
    while d > error and it < itmax:
        
        # Vector actual
        rc = r
        
        F = GetF(G,rc)
        J = GetJacobian(G,rc)
        InvJ = np.linalg.inv(J)
        
        r = rc - np.dot(InvJ,F)
        
        diff = r - rc
        
        d = np.max( np.abs(diff) )
        
        dvector.append(d)
        
        it += 1
    
    return r,dvector
r,dvector=NewtonRaphson(G2,np.array([0.,0.,0.]))

# Con descenso al gradiente 

def GetF(G,r):
    
    n = r.shape[0]
    
    v = np.zeros_like(r)
    
    for i in range(n):
        v[i] = G[i](r[0],r[1],r[2])
        
    return v

def Metric(G,r):
    return 0.5*np.linalg.norm(GetF(G,r))**2

def GetJacobian(f,r,h=1e-6):
    
    n = r.shape[0]
    
    J = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            
            rf = r.copy()
            rb = r.copy()
            
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            
            J[i,j] = ( f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2])  )/(2*h)
            
    
    return J

def Minimizer(G,r,lr=1e-2,epochs=int(1e4),error=1e-14):
    
    metric = 1
    it = 0
    
    M = np.array([])
    R = np.array([r])
    
    while metric > error and it < epochs:
        
        M = np.append(M,Metric(G,r))
        J = GetJacobian(G,r)
        Vector = GetF(G,r)
        # Machine learning
        r -= lr*np.dot(J,Vector)
        
        R = np.vstack((R,r))
        
        metric = Metric(G,r)
        
        it += 1
        
        if it%50 == 0:
            clear_output(wait=True)
            print(r)
            time.sleep(0.001)
    
    return r
xsol = Minimizer(G2,np.array([0.,0.,0.]))

print ("Las solucion usando newton Rhapson es " + str(r) + " y usando descenso al gradiente "+ str(xsol))


[ 0.4999951  -0.22600227 -0.02399521]
Las solucion usando newton Rhapson es [ 0.4999951  -0.22600226 -0.02399529] y usando descenso al gradiente [ 0.4999951  -0.22600226 -0.02399527]


In [ ]:
#punto 15

import sympy as sym 


In [27]:
# punto 16 
import sympy as sym 
from sympy import I
from sympy import Matrix

y0=Matrix([[1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,-1]])
y1=Matrix([[0,0,0,1],[0,0,1,0],[0,-1,0,0],[-1,0,0,0]])
y2=Matrix([[0,0,0,-I],[0,0,I,0],[0,I,0,0],[-I,0,0,0]])
y3=Matrix([[0,0,1,0],[0,0,0,-1],[-1,0,0,0],[0,1,0,0]])
metric=sym.diag(1,-1,-1,-1)
Identidad=2*metric*Matrix.eye(4)
#Caso 1 v=u
s0=(y0*y0*metric+y0*y0*metric)*-1
s1=(y1*y1*metric+y1*y1*metric)*-1
s2=(y2*y2*metric+y2*y2*metric)*-1
s3=(y3*y3*metric+y3*y3*metric)*-1
if s1==s2==s3==Identidad:
    print("En el caso v=u si son conmutativos")
else: 
    print("No son conmutativos")

#Cas0 2 v=u

s01=(y0*y1*metric+y1*y0*metric)*-1
s12=(y1*y2*metric+y2*y1*metric)*-1
s23=(y2*y3*metric+y3*y2*metric)*-1

if s01==s12==s23==Identidad: 
    print("El caso v=! u es conmutativo")
    
else:
    print("El caso u=!v no es conmutativo")

En el caso v=u si son conmutativos
El caso u=!v no es conmutativo


In [9]:
#Punto 17

import sympy as sym

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)
z = x + sym.I*y

def f(x,y,z):

    f = z**3 + 1
    f = f.expand()
    return sym.re(f),sym.im(f)

f_re,f_im = f(x,y,z)
F = [f_re,f_im]

J = sym.zeros(2,2)

for i in range(2):
    for j in range(2):
        if j==0:
            J[i,j] = sym.diff(F[i],x,1)
        else:
            J[i,j] = sym.diff(F[i],y,1)

J

Fn = sym.lambdify([x,y],F,'numpy')
Jn = sym.lambdify([x,y],J,'numpy')
InvJ = J.inv(method='LU')
InvJ

def raiz(z,Fn,Jn,itmax=100,precision=1e-7):
    #Se invierte el Jacobiano
    InvJ = Jn.inv(method='LU')
    IJn = sym.lambdify([x,y],InvJ,'numpy')
    InvJ
    
    error = 1
    it = 0
    
    while error > precision and it < itmax:
        IFn = Fn(z[0],z[1])
        InJn = IJn(z[0],z[1])
        z1 = z- np.dot(InJn,IFn)
        
        error = np.max( np.abs(z1-z) )
        z= z1
        it +=1
        
    return z

z0= np.array([0.5 , 0.6])
zsol= raiz(z0,Fn,J)
zsol

N= 300
x = np.linspace(-1, 1, N)
y = np.linspace(-1, 1, N)